In [ ]:
from datetime import datetime, timedelta
import os
from pathlib import Path
import pandas as pd


def duration_str(duration: timedelta):
    """
    Use total seconds to convert to a datetime and format as a string e.g. 01:30
    """
    return datetime.fromtimestamp(duration.total_seconds()).strftime("%H:%M")


DATA_DIR = Path("./data")
DATA_SOURCE = Path(os.environ.get("HARVEST_DATA", "./data/harvest-sample.csv"))

In [ ]:
# assign category dtype for efficiency on repeating text columns
dtypes = {
    "Client": "category",
    "Project": "category",
    "First Name": "category",
    "Last Name": "category",
}
# skip reading the columns we don't care about for Toggl
cols = list(dtypes) + [
    "Date",
    "Notes",
    "Hours",
]
# read CSV file, parsing dates
source = pd.read_csv(DATA_SOURCE, dtype=dtypes, usecols=cols, parse_dates=["Date"], cache_dates=True)
source.dtypes

In [ ]:
# rename columns that can be imported as-is
source.rename(columns={"Project": "Task", "Notes": "Description", "Date": "Start Date"}, inplace=True)
source.dtypes

In [ ]:
# update static calculated columns
source["Client"] = "Xentrans"
source["Client"] = source["Client"].astype("category")
source["Project"] = "Xentrans"
source["Project"] = source["Project"].astype("category")
source["Billable"] = "Yes"
source["Billable"] = source["Billable"].astype("category")
source.dtypes

In [ ]:
# add the Email column
source["Email"] = source["First Name"].apply(lambda x: f"{x.lower()}@compiler.la").astype("category")
# drop individual name columns
source.drop(columns=["First Name", "Last Name"], inplace=True)
source.dtypes

In [ ]:
# Convert numeric Hours to timedelta Duration
source["Duration"] = source["Hours"].apply(pd.to_timedelta, unit="hours")

In [ ]:
# Default start time to 09:00
source["Start Time"] = pd.to_timedelta("09:00:00")
source.dtypes

In [ ]:
def calc_start_time(group):
    """
    Start time is offset by the previous record's duration, with a default of 0 offset for the first record
    """
    group["Start Time"] = group["Start Time"] + group["Duration"].shift(fill_value=pd.to_timedelta("00:00:00")).cumsum()
    return group

In [ ]:
# sort and group users into their distinct days
source.sort_values(["Email", "Start Date"], inplace=True)
user_days = source.groupby(["Email", "Start Date"])
user_days = user_days.apply(calc_start_time)